# Searching Case Study: Nonogram

Nonogram Puzzle is a logical picture puzzle played on a grid with numerical hints on the top and the left side of the grid. The hints are used to know which cell of the grid is supposed to be colored.

Here, we introduce you the searching approach to solve this puzzle. We utilized the search problem model created by Russell And Norvig's "Artificial Intelligence - A Modern Approach" - Chapter 3.

## Getting Started

### Initializing Nonogram Board

In [25]:
from src.gen import Gen
from src.puzzle import Nonogram
from src.state import State

grid = [
    [1, 1, 1, 0, 1, 0, 1],
    [1, 1, 1, 0, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1],
    [1, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 1, 0, 1],
    [1, 0, 1, 1, 0, 0, 1],
]
init = State(len(grid), num=Gen.gen_grid_num(grid))
puzzle = Nonogram(init)

### Import Packages for Performance Testing

In [26]:
import os
import time
import psutil

def process_memory():
    return psutil.Process(os.getpid()).memory_info().rss

## Depth-First Search

Techniques used to optimize square-based DFS (which is, brute force all NxN squares):

- Level-Block model

    - The DFS traverse level by level (row or column, here we are using row)
    - Pick the blocks of size `number` x 1 from the corresponding hints (consisting of `number`s) of the level
    - Try all possible block positions within that level
    - After all level's block is picked, go to the next level

- Level pruning

    - Prune all other level from the search tree, if they have the same parent node

- Block position pruning

    - Skip the impossible block positions by simple calculations

### Import DFS algorithm

In [27]:
from src.search import DFS

### Time Elapsed

In [28]:
start_time = time.time()
solution = DFS(puzzle)
elapsed_time = time.time() - start_time

### Memory Consumption

In [29]:
start_memory = process_memory()
solution = DFS(puzzle)
consumed_memory = process_memory() - start_memory

### Output

In [30]:
print(f"Time taken: {round(elapsed_time, 6)} seconds")
print(f"Memory usage: {consumed_memory / 1024.0 / 1024.0} MB")
print(f"\nSolution:")
print(solution.path())

Time taken: 11.705693483352661 seconds
Memory usage: 16.109375 MB

Solution:
[<Node 
<State level=None remain={0, 1, 2, 3, 4, 5, 6}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 

0  0  0  0  0  0  0  

>
>, <Node 
<State level=6 remain={0, 1, 2, 3, 4, 5}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 

0  0  0  0  0  0  0  

>
>, <Node 
<State level=6 remain={0, 1, 2, 3, 4, 5}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  #  .  .  .  .  .    1 

0  1  0  0  0  0  0  

>
>, <Node 
<State level=6 remain={0, 1, 2, 3, 4, 5}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .

## Best-First Search

Techniques used to implement Best First Search, in order to speed up the proposed DFS:

- Level and block position pruning (mentioned above)
- Transform `frontier` from a stack to a priority queue, with priority measured using a heuristic function
- Consider some possible heuristic functions:

    - Level heuristic: Prioritize levels with the largest block (secondly, a maximum total number of size)
    - Column heuristic: Prioritize block position with minimum conflicts (experimental)

### Import Best First Search Algorithm and the Heuristic Function

In [31]:
from src.search import BeFS
from src.utils import heuristic_level

### Time Elapsed

In [32]:
start_time = time.time()
solution = BeFS(puzzle,heuristic_level)
elapsed_time = time.time() - start_time

### Memory Consumption

In [33]:
start_memory = process_memory()
solution = BeFS(puzzle, heuristic_level)
consumed_memory = process_memory() - start_memory

### Output

In [34]:
print(f"Time taken: {round(elapsed_time, 6)} seconds")
print(f"Memory usage: {consumed_memory / 1024.0 / 1024.0} MB")
print(f"\nSolution:")
print(solution.path())

Time taken: 0.8243846893310547 seconds
Memory usage: 0.01171875 MB

Solution:
[<Node 
<State level=None remain={0, 1, 2, 3, 4, 5, 6}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 

0  0  0  0  0  0  0  

>
>, <Node 
<State level=2 remain={0, 1, 3, 4, 5, 6}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 

0  0  0  0  0  0  0  

>
>, <Node 
<State level=2 remain={0, 1, 3, 4, 5, 6}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  #  #  #  #  #    5 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 

0  0  1  1  1  1  1  

>
>, <Node 
<State level=0 remain={1, 3, 4, 5, 6}
.  .  .  .  .  .  .    0 
.  .  .  .  .  .  .    0 
.  .  #  #  #  #  #    5 
.  .  .  .  .  